In [1]:
import pandas as pd
import numpy as np 
from gensim import models
from lib import weat
from lib import weat_main
import os
import scipy
import matplotlib.pyplot as plt
import importlib
from scipy import stats
import seaborn as sns
import fasttext
import fasttext.util

### Read the CMU speech transcripts

In [88]:
import wave

In [85]:
txt_file = '/Users/adimaini/Documents/GW/Research/CODE.nosync/WEAT-WEFAT/data/cmu_haitian_speech/etc/txt.done.data'

transcript = pd.read_csv(txt_file, sep='"', header=None).drop(columns=[2])
transcript.columns = ['id', 'text']

In [86]:
# clean out the 'id' and 'text' column spacing and extra paranthesis
transcript['id'] = transcript['id'].apply(lambda x: x.lstrip('( ').strip())
transcript['text'] = transcript['text'].apply(lambda x: x.lstrip().strip())

# sort the transcripts in a certain manner
# transcript = transcript.sort_values(ascending=False, by='id').reset_index(drop=True)

# shuffle the transcripts to allow for random speakers for our accuracy sub sample
transcript = transcript.sample(frac=1, random_state=5)

### Read the CMU audio files and order them in the same fashion as the transcript data

In [113]:
# Read the CMU audio files and order them in the same fashion as the transcript data
path_to_wav = '/Users/adimaini/Documents/GW/Research/CODE.nosync/WEAT-WEFAT/data/cmu_haitian_speech/wav/'
dir_file = os.listdir(path=path_to_wav)

In [114]:
# output the combined .wav files. in order to match the duration of the radio transcription, 
# we will only select 200 transcripts (31 minutes) of the transcripts at first. 

outfile = 'data/cmu_haitian_speech/aws/cmu_combined.wav'
no_samples = 180

data = []
for t_id in transcript.id[:no_samples]: 
    file = path_to_wav + t_id + '.wav'
    w = wave.open(file, 'rb')
    data.append( [w.getparams(), w.readframes(w.getnframes())] )
    w.close()
    
output = wave.open(outfile, 'wb')
output.setparams(data[0][0])
for i in range(len(data)): 
    output.writeframes(data[i][1])
output.close()

In [110]:
text = ' '.join(transcript[:no_samples].text.values)

In [111]:
text

'Konsa tou, sa nou wè koulye a, se tankou yon pòtrè nap gade yon jan twoub twoub nan yon glas. Avantaj baraj Pelig la t ap bay peyizan Latibonit yo chans pou jwenn dlo pandan tout sezon an. Kouman ou ka aji ak moun tankou si yo te pwason nan lanmè, tankou koulèv nan raje ki pa gen mèt? Parabòl ja lajan an Peyi Wa ki nan syèl la, se tankou yon ja lajan ki te anba tè nan yon jaden. Aprè sa, li pale avèk sa ki sou bò gòch li yo, li di: wete kò nou sou mwen, nou menm ki gen madichon. Epi li rete konsa li wè gwo lafimen ap moute sòti nan tè a tankou lafimen k ap leve nan yon gwo founo dife. Epi li rete konsa li wè gwo lafimen ap moute sòti nan tè a tankou lafimen k ap leve nan yon gwo founo dife. Se pou l tounen dènye klas esklav frè l yo. Lè sa a, se wa Akaz, pitit gason Jotam, pitit pitit gason Ozyas, ki t ap gouvène nan peyi Jida. Se li ki sèvi m ranpa, se li ki tout delivrans mwen, se li ki pwoteksyon mwen, se anba zèl li mwen kache. Dirijan politik lavalas yo tankou bon koutye malfektè

In [115]:
import json

In [120]:
aws_file = '/Users/adimaini/Documents/GW/Research/CODE.nosync/WEAT-WEFAT/data/cmu_haitian_speech/aws/asrOutput.json'
f = open(aws_file)

aws_output = json.load(f)

In [133]:
aws_transcript = aws_output['results']['transcripts'][0]['transcript']

In [134]:
!pip install jiwer

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 50 kB 858 kB/s eta 0:00:01
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-macosx_10_14_x86_64.whl size=81520 sha256=e4784d8f9d1791bbbff98460ad0c502217f1e9e506a05a7b4c0c30aaab046a63
  Stored in directory: /Users/adimaini/Library/Caches/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [135]:
from jiwer import wer
error = wer(text, aws_transcript)

In [136]:
error

0.988116458704694